# Intro to Obspy

Obspy is a Python package for seismology. It contains a wide range of tools, from fundamental input/output (IO) functions, up to higher
level functions including noise analysis, beamforming and wrappers for travel-time calculations.  Obspy has rapidly gained popularity,
as of February 2019, Obspy version 1.1.0 had been installed from conda-forge 11,720 times in the 3 months that it was the current version.

## Python

<img alt="Python logo" align="right" style="width:40%" src="https://www.python.org/static/community_logos/python-logo-master-v3-TM.png">

Python as a language is widely used, both within the scientific community and in more applied work. It is also open-source, meaning that
the licence limitations of something like Matlab are not an issue.  Because Python is widely used there are some fantastic libraries,
meaning that you do not need to re-invent the wheel - someone has probably solved your problem already - meaning you can focus on
doing science. Furthermore, because Python is used beyond academia, it is a useful, transferable language to learn.

Python is an interpreted language, meaning that you do not need to actively compile code to machine code before executing your core
(unlike languages like C or Fortran). This means that you can rapidly test short ideas, and allows us to make use of things like
Jupyter notebooks and the iPython interpretter.  In general interpreted languages are slower than compilled languages, and this
is generally true for Python as well.  However, because Python is simple to extend with compiled languages (particularly C), Python
provide a good *glue* between faster code components.  It is normal to convert time-critical sections of code to C and link these
to a Python library. Numpy makes good use of this, and links some very fast compiled maths libraries (LAPACK and BLAS) in to Python.

## Obspy

<img alt="Obspy logo" align="right" style="width:40%" src="https://raw.github.com/obspy/website/master/logo/obspy_logo_full_highres.png">

Obspy is actively maintained, developed and tested by an international group of seismologists.  Obspy also implements a good 
*continuous integration* workflow, whereby any proposed changes to the package are tested before they are accepted. The complete
package is also tested prior to release, ensuring that Obspy does what it is supposed to. This kind of testing is incredibly
important, and, while it does not guarantee correctness, it is a very necessary step to test code to ensure that it does what
you expect it to.  Obspy also has fairly extensive documentation and good examples and tutorials.

This notebook provides a really brief overview of some of the "every-day" functionality of Obspy that you will use within this
course.  We will cover:
- Reading in seismic data
- Plotting seismic data
- Filtering seismic data
- Getting data from data centres
- Plotting spectrograms
- Reading in event data
- Plotting earthquake maps
- Accesing relevant event information within a catalog